In [1]:
# mypy: allow-untyped-defs
"""
This module implements Paged Attention on top of flex_attention.
This module is experimental and subject to change.
"""

import torch
from torch.nn.attention.flex_attention import (
    _identity,
    _mask_mod_signature,
    _score_mod_signature,
    BlockMask,
    noop_mask,
    flex_attention,
    create_block_mask,
)

__all__ = ["PagedAttention"]


def _cdiv(x: int | float | torch.Tensor, multiple: int | float | torch.Tensor):
    return (x + multiple - 1) // multiple


class PagedAttention:
    """
    PagedAttention supports flex attention inference with a large batch size.
    With PagedAttention, a batch of key/value tensors with varying kv length
    is split into tensor blocks of fixed length and cached in a compact way.
    Thus we can avoid redundant memory consumption due to varying kv length and
    support a larger batch size.
    """

    def __init__(
        self,
        n_pages: int,
        page_size: int,
        max_batch_size: int,
        device: str = "cuda",
    ) -> None:
        # number of pages
        self.n_pages = n_pages

        # number of tokens per page
        self.page_size = page_size

        # page table: [batch, logical_block_idx] -> physical_page_idx
        self.page_table = -torch.ones(
            (max_batch_size, self.n_pages), dtype=torch.int64, device=device
        )

        # capacity: batch_idx -> allocated sequence length
        self.capacity = torch.zeros(max_batch_size, dtype=torch.int64, device=device)

        # index of empty pages that is available for allocation
        self.empty_pages = list(range(n_pages - 1, -1, -1))

        # mapping from physical page index to logical page index
        self.physical_to_logical = -torch.ones(
            (max_batch_size, n_pages), dtype=torch.int64, device=device
        )
    def reserve(self, batch_idx: int, seq_len: torch.Tensor) -> None:
      seq_len = int(seq_len.item())
      cur_capacity = int(self.capacity[batch_idx].item())  # 🔴 FIX

      if seq_len <= cur_capacity:
          return

      needed = seq_len - cur_capacity
      num_pages = (needed + self.page_size - 1) // self.page_size

      if len(self.empty_pages) < num_pages:
          raise RuntimeError(
              f"requested {num_pages} pages but only {len(self.empty_pages)} available"
          )

      start_page = cur_capacity // self.page_size
      end_page = start_page + num_pages

      allocated_pages = torch.tensor(
          self.empty_pages[-num_pages:],
          device=self.page_table.device,
          dtype=torch.int64,
      )
      self.empty_pages = self.empty_pages[:-num_pages]

      self.page_table[batch_idx, start_page:end_page] = allocated_pages
      self.physical_to_logical[batch_idx, allocated_pages] = torch.arange(
          start_page, end_page, device=self.page_table.device
      )

      self.capacity[batch_idx] += num_pages * self.page_size


    # def reserve(self, batch_idx: torch.Tensor, seq_len: torch.Tensor) -> None:
    #     """
    #     Requests the capacity of a given batch to be at least enough to
    #     hold `seq_len` elements.

    #     Args:
    #         batch_idx (Tensor): batch index to be reserved; shape :math:`(1)`.
    #         seq_len (Tensor): minimum capacity for the given batch; shape :math:`(1)`.
    #     """

    #     if seq_len <= self.capacity[batch_idx]:
    #         return

    #     num_pages_to_allocate = _cdiv(
    #         seq_len - self.capacity[batch_idx], self.page_size
    #     )

    #     if len(self.empty_pages) < num_pages_to_allocate:
    #         raise AssertionError(
    #             f"requested {num_pages_to_allocate.item()} pages "
    #             f"but there are only {len(self.empty_pages)} empty pages"
    #         )

    #     start_page_idx = self.capacity[batch_idx] // self.page_size
    #     end_page_idx = start_page_idx + num_pages_to_allocate

    #     # find empty physical pages
    #     allocated_pages = torch.tensor(
    #         self.empty_pages[-num_pages_to_allocate:],
    #         device=num_pages_to_allocate.device,
    #     )
    #     self.empty_pages = self.empty_pages[:-num_pages_to_allocate]

    #     # update page table
    #     self.page_table[
    #         batch_idx,
    #         start_page_idx:end_page_idx,
    #     ] = allocated_pages

    #     # update metadata
    #     self.physical_to_logical[batch_idx, allocated_pages] = torch.arange(
    #         start_page_idx.item(),
    #         end_page_idx.item(),
    #         device=num_pages_to_allocate.device,
    #     )
    #     self.capacity[batch_idx] += num_pages_to_allocate * self.page_size
    def erase(self, batch_idx: int) -> None:
      allocated = self.page_table[batch_idx] != -1
      pages = self.page_table[batch_idx][allocated]

      self.capacity[batch_idx] = 0
      self.empty_pages += pages.tolist()
      self.physical_to_logical[batch_idx].fill_(-1)
      self.page_table[batch_idx].fill_(-1)

    # def erase(self, batch_idx: torch.Tensor) -> None:
    #     """
    #     Removes a single batch from paged attention.

    #     Args:
    #         batch_idx (Tensor): batch index to be removed; shape :math:`(1)`.
    #     """

    #     # find allocated pages
    #     allocated_page_idx = self.page_table[batch_idx] != -1
    #     allocated_pages = self.page_table[batch_idx][allocated_page_idx]

    #     # clean metadata
    #     self.capacity[batch_idx] = 0
    #     self.empty_pages += allocated_pages.tolist()
    #     self.physical_to_logical[batch_idx][:, allocated_pages] = -1
    #     self.page_table[batch_idx] = -1
    def assign(
      self,
      batch_idx: torch.Tensor,
      input_pos: torch.Tensor,
      k_val: torch.Tensor,
      v_val: torch.Tensor,
      k_cache: torch.Tensor,
      v_cache: torch.Tensor,
  ) -> None:

      if k_val.requires_grad:
          raise RuntimeError("k_val must not require grad")

      B, H, S, K_D = k_val.shape
      V_D = v_val.shape[-1]

      # --- logical → physical mapping ---
      logical_block = input_pos // self.page_size
      logical_offset = input_pos % self.page_size

      # 🔴 FIX: batch-wise gather
      page_table = self.page_table[batch_idx]              # [B, n_pages]
      physical_block = torch.gather(
          page_table, 1, logical_block.to(torch.int64)
      )

      # --- SAFETY CHECK ---
      if (physical_block < 0).any():
          raise RuntimeError("assign() called before reserve()")

      addr = (physical_block * self.page_size + logical_offset).reshape(-1)

      # flatten KV
      k_val = k_val.permute(1, 0, 2, 3).reshape(1, H, B * S, K_D)
      v_val = v_val.permute(1, 0, 2, 3).reshape(1, H, B * S, V_D)

      k_cache[:, :, addr, :] = k_val
      v_cache[:, :, addr, :] = v_val

    # def assign(
    #     self,
    #     batch_idx: torch.Tensor,
    #     input_pos: torch.Tensor,
    #     k_val: torch.Tensor,
    #     v_val: torch.Tensor,
    #     k_cache: torch.Tensor,
    #     v_cache: torch.Tensor,
    # ) -> None:
    #     """
    #     Assigns new contents `val` to the storage `cache` at the location
    #     `batch_idx` and `input_pos`.

    #     Args:
    #         batch_idx (Tensor): batch index; shape :math:`(B)`.
    #         input_pos (Tensor): input positions to be assigned for the given batch; shape :math:`(B, S)`.
    #         val (Tensor): value to be assigned; shape :math:`(B, H, S, D)`
    #         cache (Tensor): the cache to store the values; shape:`(1, H, MAX_S, D)`
    #     """
    #     if k_val.requires_grad:
    #         raise RuntimeError("val must not require gradient")

    #     B, H, S, K_D = k_val.shape
    #     V_D = v_val.shape[3]
    #     if B != batch_idx.shape[0]:
    #         raise RuntimeError(
    #             f"Expect val and batch_idx have the same batch size "
    #             f"but got B={B} and B={batch_idx.shape[0]}."
    #         )
    #     if H != k_cache.shape[1]:
    #         raise RuntimeError(
    #             f"Expect val and cache has the same number of heads "
    #             f"but got H={H} and H={k_cache.shape[1]}."
    #         )
    #     if S != input_pos.shape[1]:
    #         raise RuntimeError(
    #             f"Expect val and input_pos has the same length "
    #             f"but got S={S} and S={input_pos.shape[0]}."
    #         )
    #     if K_D != k_cache.shape[3]:
    #         raise RuntimeError(
    #             f"Expect k_val and k_cache has the same hidden dim "
    #             f"but got D={K_D} and D={k_cache.shape[3]}."
    #         )
    #     if V_D != v_cache.shape[3]:
    #         raise RuntimeError(
    #             f"Expect v_val and v_cache has the same hidden dim "
    #             f"but got D={V_D} and D={v_cache.shape[3]}."
    #         )

    #     # find address
    #     logical_block_idx = input_pos // self.page_size  # [B, S]
    #     logical_block_offset = input_pos % self.page_size  # [B, S]
    #     physical_block_idx = torch.gather(
    #         self.page_table[batch_idx], 1, logical_block_idx.to(torch.int64)
    #     ).to(torch.int32)  # [B, S]

    #     addr = (physical_block_idx * self.page_size + logical_block_offset).view(
    #         -1
    #     )  # [B*S]

    #     k_val = k_val.permute(1, 0, 2, 3).contiguous().view(1, H, B * S, K_D)
    #     v_val = v_val.permute(1, 0, 2, 3).contiguous().view(1, H, B * S, V_D)

    #     k_cache[:, :, addr, :] = k_val
    #     v_cache[:, :, addr, :] = v_val

    def convert_logical_block_mask(
        self,
        block_mask: BlockMask,
        batch_idx: torch.Tensor | None = None,
        kv_len: torch.Tensor | None = None,
    ) -> BlockMask:
        """
        Converts a logical block mask by mapping its logical kv indices to the corresponding
        physical kv indices.

        Args:
            block_mask (BlockMask): logical block mask;
                kv_indices shape :math:`(B, H, ROWS, MAX_BLOCKS_IN_COL)`.
            batch_idx (Tensor): batch index corresponding to the block_mask
                batch dimension. This provides flexibility to convert a
                block mask with smaller batch size than the page table;
                shape :math:`(B)`.
            kv_len (Optional[Tensor]): actual KV sequence length for upper bound check;
                shape :math:`(B,)` to handle multiple batches.
        """
        B, H, ROWS, MAX_BLOCKS_IN_COL = block_mask.kv_indices.shape

        if block_mask.BLOCK_SIZE[1] != self.page_size:
            raise RuntimeError(
                f"Expect block_mask has the same column block size as page_size"
                f"but got size={block_mask.BLOCK_SIZE[1]} and size={self.page_size}"
            )

        device = block_mask.kv_num_blocks.device

        if batch_idx is None:
            batch_idx = torch.arange(B, device=device)
        page_table = self.page_table[batch_idx]

        new_kv_num_blocks = block_mask.kv_num_blocks.clone()

        # The physical page table might be larger than the max logical blocks
        # but the block mask only cares about MAX_BLOCKS_IN_COL.
        # We assume the logical mask is sparse enough.
        new_kv_indices = torch.zeros(
            (B, H, ROWS, self.n_pages), dtype=torch.int32, device=device
        )

        # We gather the physical page indices using the logical block indices
        # provided by the input block_mask.
        new_kv_indices[:, :, :, :MAX_BLOCKS_IN_COL] = (
            torch.gather(
                page_table, 1, block_mask.kv_indices.view(B, -1).to(torch.int64)
            )
            .view(block_mask.kv_indices.shape)
            .to(torch.int32)
        )

        new_full_kv_indices, new_full_kv_num_blocks = None, None
        if block_mask.full_kv_num_blocks is not None:
            if block_mask.full_kv_indices is None:
                raise AssertionError(
                    "block_mask.full_kv_indices must not be None when full_kv_num_blocks is not None"
                )
            new_full_kv_num_blocks = block_mask.full_kv_num_blocks.clone()
            new_full_kv_indices = torch.zeros(
                (B, H, ROWS, self.n_pages), dtype=torch.int32, device=device
            )
            new_full_kv_indices[:, :, :, :MAX_BLOCKS_IN_COL] = (
                torch.gather(
                    page_table,
                    1,
                    block_mask.full_kv_indices.view(B, -1).to(torch.int64),
                )
                .view(block_mask.full_kv_indices.shape)
                .to(torch.int32)
            )

        new_mask_mod = self.get_mask_mod(block_mask.mask_mod, kv_len)

        # The K/V cache sent to flex_attention acts as one giant sequence of length n_pages * page_size.
        seq_lengths = (block_mask.seq_lengths[0], self.n_pages * self.page_size)

        return BlockMask.from_kv_blocks(
            new_kv_num_blocks,
            new_kv_indices,
            new_full_kv_num_blocks,
            new_full_kv_indices,
            block_mask.BLOCK_SIZE,
            new_mask_mod,
            seq_lengths=seq_lengths,
        )

    def get_mask_mod(
        self,
        mask_mod: _mask_mod_signature | None,
        kv_len: torch.Tensor | None = None,
    ) -> _mask_mod_signature:
        """
        Converts a mask_mod based on mapping from the physical block index to the logical
        block index.
        """
        if mask_mod is None:
            mask_mod = noop_mask

        def new_mask_mod(
            b: torch.Tensor,
            h: torch.Tensor,
            q_idx: torch.Tensor,
            physical_kv_idx: torch.Tensor,
        ):
            physical_kv_block = physical_kv_idx // self.page_size
            physical_kv_offset = physical_kv_idx % self.page_size
            logical_block_idx = self.physical_to_logical[b, physical_kv_block]
            logical_kv_idx = logical_block_idx * self.page_size + physical_kv_offset
            live_block = logical_block_idx >= 0
            within_upper_bound = (
                logical_kv_idx < kv_len[b] if kv_len is not None else True
            )
            within_lower_bound = logical_kv_idx >= 0
            is_valid = live_block & within_upper_bound & within_lower_bound

            return torch.where(is_valid, mask_mod(b, h, q_idx, logical_kv_idx), False)

        return new_mask_mod

    # def get_score_mod(
    #     self,
    #     score_mod: _score_mod_signature | None,
    #     kv_len: torch.Tensor | None = None,
    # ) -> _score_mod_signature:
    #     """
    #     Converts a score_mod based on mapping from the physical block index to the logical
    #     block index.
    #     """
    #     if score_mod is None:
    #         score_mod = _identity

    #     def new_score_mod(
    #         score: torch.Tensor,
    #         b: torch.Tensor,
    #         h: torch.Tensor,
    #         q_idx: torch.Tensor,
    #         physical_kv_idx: torch.Tensor,
    #     ):
    #         physical_kv_block = physical_kv_idx // self.page_size
    #         physical_kv_offset = physical_kv_idx % self.page_size
    #         logical_block_idx = self.physical_to_logical[b, physical_kv_block]
    #         logical_kv_idx = logical_block_idx * self.page_size + physical_kv_offset
    #         live_block = logical_block_idx >= 0
    #         within_upper_bound = (
    #             logical_kv_idx < kv_len[b] if kv_len is not None else True
    #         )
    #         within_lower_bound = logical_kv_idx >= 0
    #         is_valid = live_block & within_upper_bound & within_lower_bound

    #         return torch.where(
    #             is_valid,
    #             score_mod(score, b, h, q_idx, logical_kv_idx),
    #             float("-inf"),
    #         )

    #     return new_score_mod
    def get_score_mod(
        self,
        score_mod: _score_mod_signature | None,
        kv_len: torch.Tensor | None = None,
        is_causal: bool = True, # Add this flag
    ) -> _score_mod_signature:
        if score_mod is None:
            score_mod = _identity

        def new_score_mod(
            score: torch.Tensor,
            b: torch.Tensor,
            h: torch.Tensor,
            q_idx: torch.Tensor,
            physical_kv_idx: torch.Tensor,
        ):
            # 1. Map Physical to Logical
            physical_kv_block = physical_kv_idx // self.page_size
            physical_kv_offset = physical_kv_idx % self.page_size
            logical_block_idx = self.physical_to_logical[b, physical_kv_block]
            logical_kv_idx = logical_block_idx * self.page_size + physical_kv_offset

            # 2. VALIDITY CHECKS
            # Block must be allocated to this batch
            live_block = logical_block_idx >= 0
            # Must be within the actual sequence length (ignore garbage in the rest of the page)
            within_bounds = (logical_kv_idx < kv_len[b]) if kv_len is not None else True
            # Causal check (MUST use logical indices)
            causal_ok = (q_idx >= logical_kv_idx) if is_causal else True

            is_valid = live_block & within_bounds & causal_ok

            return torch.where(
                is_valid,
                score_mod(score, b, h, q_idx, logical_kv_idx),
                float("-inf"),
            )

        return new_score_mod

    def forward(
        self,
        query: torch.Tensor,
        k_cache: torch.Tensor,
        v_cache: torch.Tensor,
        kv_lens: torch.Tensor,
        block_mask: BlockMask | None = None,
        batch_indices: torch.Tensor | None = None,
        score_mod: _score_mod_signature | None = None,
        scale: float | None = None,
        enable_gqa: bool = False,
    ):
        """
        Computes the paged attention.

        Args:
            query (Tensor): Query tensor; shape :math:`(B, H, Q_LEN, D)`.
            k_cache (Tensor): Physical key cache; shape :math:`(1, H, TOTAL_PAGES * PAGE_SIZE, D)`.
            v_cache (Tensor): Physical value cache; shape :math:`(1, H, TOTAL_PAGES * PAGE_SIZE, D)`.
            kv_lens (Tensor): The logical length of the KV sequence for each batch element.
                              Used to mask out padding and future tokens. Shape :math:`(B,)`.
            block_mask (BlockMask, optional): Logical block mask. If None, a standard causal mask is assumed
                                              for the logical sequence.
            batch_indices (Tensor, optional): The indices in the page table corresponding to the query batch.
                                              If None, assumes 0..B-1.
            score_mod (callable, optional): Logical score modification function.
            scale (float, optional): Attention scale factor.
            enable_gqa (bool): Enable Grouped Query Attention.
        """
        B, H, Q_LEN, D = query.shape
        device = query.device

        if batch_indices is None:
            batch_indices = torch.arange(B, device=device)

        # 1. Create a logical block mask if one isn't provided.
        # We assume a standard Causal mask over the maximum logical KV length.
        if block_mask is None:
            max_kv_len = kv_lens.max().item()

            def causal_mask(b, h, q, k):
                return q >= k

            # Create a block mask for the logical dimensions: (B, H, Q_LEN, Max_KV)
            # Note: We must ensure the BLOCK_SIZE matches the page_size.
            block_mask = create_block_mask(
                causal_mask,
                B=B,
                H=H,
                Q_LEN=Q_LEN,
                KV_LEN=max_kv_len,
                device=device,
                BLOCK_SIZE=self.page_size,
                _compile=True # Often improves performance for mask creation
            )

        # 2. Convert the logical block mask to a physical block mask.
        # This re-maps the indices in the block mask to point to the correct physical pages.
        physical_block_mask = self.convert_logical_block_mask(
            block_mask,
            batch_idx=batch_indices,
            kv_len=kv_lens
        )

        # 3. Handle Score Mods
        # Wrap the user score_mod (which expects logical indices) to handle physical indices.
        physical_score_mod = self.get_score_mod(score_mod, kv_lens)

        # 4. Call flex_attention
        # Note: k_cache and v_cache have batch size 1, but query has batch size B.
        # The block mask handles the routing (preventing batch B from seeing batch A's pages).
        output = flex_attention(
            query,
            k_cache,
            v_cache,
            block_mask=physical_block_mask,
            score_mod=physical_score_mod,
            scale=scale,
            enable_gqa=enable_gqa
        )

        return output

In [2]:
import torch
import torch.nn.functional as F
from torch.nn.attention.flex_attention import create_block_mask

# Assuming the PagedAttention class is defined in the same file or imported
# from paged_attention import PagedAttention

def test_paged_attention_correctness():
    print("=== Starting Paged Attention Test ===")

    if not torch.cuda.is_available():
        print("Skipping test: CUDA not available (FlexAttention requires CUDA).")
        return

    device = "cuda"
    torch.set_default_device(device)

    # --- 1. Configuration ---
    BATCH_SIZE = 4
    NUM_HEADS = 6
    HEAD_DIM = 64
    PAGE_SIZE = 16
    N_PAGES = 100  # Total physical pages available
    Q_LEN = 12     # Query length (e.g., chunked prefill or decoding steps)

    # Initialize Paged Attention Manager
    pa = PagedAttention(
        n_pages=N_PAGES,
        page_size=PAGE_SIZE,
        max_batch_size=BATCH_SIZE,
        device=device
    )

    # Allocate Physical Cache (Pre-allocated memory on GPU)
    # Shape: [1, H, Total_Pages * Page_Size, D]
    # Flex attention expects the cache to be treated as a single flattened sequence
    physical_k_cache = torch.zeros(1, NUM_HEADS, N_PAGES * PAGE_SIZE, HEAD_DIM, device=device)
    physical_v_cache = torch.zeros(1, NUM_HEADS, N_PAGES * PAGE_SIZE, HEAD_DIM, device=device)

    # --- 2. Generate Random Data (Ragged Batch) ---
    # We will create random KV sequences of different lengths for each batch item.
    # We will verify correctness by comparing against standard PyTorch attention.

    # Random lengths between 30 and 150 (spanning multiple pages)
    # Ensure they are > Q_LEN for this test to be interesting
    kv_lengths = torch.randint(Q_LEN + 5, 80, (BATCH_SIZE,), device=device)

    print(f"Test Configuration:")
    print(f"  Batch Size: {BATCH_SIZE}")
    print(f"  KV Lengths: {kv_lengths.tolist()}")
    print(f"  Query Len:  {Q_LEN}")
    print(f"  Page Size:  {PAGE_SIZE}")

    queries = torch.randn(BATCH_SIZE, NUM_HEADS, Q_LEN, HEAD_DIM, device=device)

    # Store ground truth K and V to run reference implementation later
    ground_truth_ks = []
    ground_truth_vs = []

    # --- 3. Populate Paged Attention ---
    print("\nPopulating Page Table and Cache...")

    for i in range(BATCH_SIZE):
        seq_len = kv_lengths[i].item()

        # Generate random K/V for this sequence
        k_data = torch.randn(1, NUM_HEADS, seq_len, HEAD_DIM, device=device)
        v_data = torch.randn(1, NUM_HEADS, seq_len, HEAD_DIM, device=device)

        ground_truth_ks.append(k_data.squeeze(0))
        ground_truth_vs.append(v_data.squeeze(0))

        # 1. Reserve pages
        batch_idx_tensor = torch.tensor([i], device=device)
        seq_len_tensor = torch.tensor([seq_len], device=device)
        pa.reserve(batch_idx_tensor, seq_len_tensor)

        # 2. Assign data
        # We assign the whole sequence at once for this test.
        # assign expects input_pos of shape [B, S]
        input_pos = torch.arange(0, seq_len, device=device).unsqueeze(0) # [1, S]

        pa.assign(
            batch_idx=batch_idx_tensor,
            input_pos=input_pos,
            k_val=k_data, # [1, H, S, D]
            v_val=v_data, # [1, H, S, D]
            k_cache=physical_k_cache,
            v_cache=physical_v_cache
        )

    # --- 4. Run Paged Attention Forward Pass ---
    print("Running Paged Attention Forward...")

    # Flex Attention scaling default is 1/sqrt(D)
    scale = 1.0 / (HEAD_DIM ** 0.5)

    # We use a standard Causal Mask implicitly handled inside the PagedAttention.forward
    # when block_mask is None.
    # Note: In a real "prefill+decoding" scenario, Q usually aligns with the END of K.
    # Here, we are simulating that Q is attending to the *entire* K history provided.

    paged_output = pa.forward(
        query=queries,
        k_cache=physical_k_cache,
        v_cache=physical_v_cache,
        kv_lens=kv_lengths,
        scale=scale
    )

    # --- 5. Run Reference Implementation (SDPA) ---
    print("Running Reference (Standard SDPA)...")

    max_diff = 0.0

    for i in range(BATCH_SIZE):
        q_i = queries[i]      # [H, Q, D]
        k_i = ground_truth_ks[i] # [H, S, D]
        v_i = ground_truth_vs[i] # [H, S, D]

        # Standard Scaled Dot Product Attention
        # We need a causal mask.
        # In this specific test setup:
        # Query (Length Q) attends to Key (Length S).
        # Since we generated S > Q, and usually Q represents the *newest* tokens,
        # we need to define the causal relationship.
        #
        # For simplicity in this test, we will assume standard broadcasting causal mask
        # is NOT applied rigidly because S != Q. We simply want to attend to all available keys
        # up to the length defined.
        # HOWEVER, PagedAttention default mask in previous code was: `q_idx >= k_idx`.
        # This implies standard causal masking.

        # Let's align reference with the Logic in PagedAttention:
        # q_idx (0..Q) attends to k_idx (0..S). Mask is True if q_idx >= k_idx.

        # Create explicit mask for SDPA
        # Shape: [Q, S]
        q_idx = torch.arange(Q_LEN, device=device).unsqueeze(1)
        k_idx = torch.arange(kv_lengths[i].item(), device=device).unsqueeze(0)
        attn_mask = (q_idx >= k_idx) # Boolean mask

        # SDPA expects float mask for add (0, -inf) or boolean (True=Keep, False=Drop)
        # But torch.nn.functional.scaled_dot_product_attention handles boolean is_causal
        # ONLY if S == Q. Since S != Q, we pass explicit mask.

        ref_out = F.scaled_dot_product_attention(
            q_i.unsqueeze(0),
            k_i.unsqueeze(0),
            v_i.unsqueeze(0),
            attn_mask=attn_mask.unsqueeze(0).unsqueeze(0),
            scale=scale
        )

        ref_out = ref_out.squeeze(0) # [H, Q, D]

        # Compare
        pa_out_i = paged_output[i]

        diff = (ref_out - pa_out_i).abs().max().item()
        max_diff = max(max_diff, diff)

        if diff > 1e-3:
            print(f"Batch {i} FAILED. Max Diff: {diff}")
        else:
            print(f"Batch {i} PASSED. Max Diff: {diff:.6f}")

    print(f"\nTest Complete. Maximum discrepancy across batch: {max_diff:.6f}")

    if max_diff < 1e-3:
        print("SUCCESS: Paged Attention matches Reference Implementation.")
    else:
        print("FAILURE: Differences detected.")

if __name__ == "__main__":
    # Ensure PagedAttention class is available here
    test_paged_attention_correctness()

=== Starting Paged Attention Test ===
Test Configuration:
  Batch Size: 4
  KV Lengths: [18, 78, 56, 63]
  Query Len:  12
  Page Size:  16

Populating Page Table and Cache...
Running Paged Attention Forward...


/usr/local/lib/python3.12/dist-packages/torch/nn/attention/flex_attention.py:1687: UserWarning: flex_attention called without torch.compile() - this will use an unfused implementation that materializes the full scores matrix instead of generating a fused kernel.

SOLUTION: Use torch.compile(flex_attention)(...)

If you want to debug your score_mod/mask_mod, you can set:
torch.nn.attention.flex_attention._FLEX_ATTENTION_DISABLE_COMPILE_DEBUG = True

This will allow you to use print statements or breakpoints. Note: This doesn't work with the backwards pass and may produce incorrect results.
  _warn_once(


Running Reference (Standard SDPA)...
Batch 0 PASSED. Max Diff: 0.000001
Batch 1 PASSED. Max Diff: 0.000001
Batch 2 PASSED. Max Diff: 0.000001
Batch 3 PASSED. Max Diff: 0.000001

Test Complete. Maximum discrepancy across batch: 0.000001
SUCCESS: Paged Attention matches Reference Implementation.


In [3]:
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained(
    "ibm-granite/granite-4.0-micro",
    torch_dtype=torch.float16,
    device_map="cuda"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
model.eval()

GraniteMoeHybridForCausalLM(
  (model): GraniteMoeHybridModel(
    (embed_tokens): Embedding(100352, 2560, padding_idx=100256)
    (layers): ModuleList(
      (0-39): 40 x GraniteMoeHybridDecoderLayer(
        (input_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
        (post_attention_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
        (shared_mlp): GraniteMoeHybridMLP(
          (activation): SiLUActivation()
          (input_linear): Linear(in_features=2560, out_features=16384, bias=False)
          (output_linear): Linear(in_features=8192, out_features=2560, bias=False)
        )
        (self_attn): GraniteMoeHybridAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=512, bias=False)
          (v_proj): Linear(in_features=2560, out_features=512, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
        )
      )
    )
    (norm): G

In [5]:
# import torch
# from torch import nn
# from torch.nn.attention.flex_attention import flex_attention
# from transformers.models.llama.modeling_llama import apply_rotary_pos_emb
# from typing import Optional, Tuple

# class PagedGraniteAttention(nn.Module):
#     def __init__(self, base_attn, config, num_pages, page_size, max_batch_size, device='cuda'):
#         super().__init__()
#         self.base_attn = base_attn
#         self.config = config
#         self.paged_attn = PagedAttention(
#             n_pages=num_pages, page_size=page_size,
#             max_batch_size=max_batch_size, device=device
#         )

#         # Granite 4.0 Micro Dimensions
#         self.head_dim = config.hidden_size // config.num_attention_heads
#         self.num_heads = config.num_attention_heads
#         self.num_kv_heads = config.num_key_value_heads

#         # Scaling factor
#         self.scaling = getattr(base_attn, "scaling", self.head_dim**-0.5)

#         # Physical Caches
#         self.k_cache = torch.zeros(
#             1, self.num_kv_heads, num_pages * page_size, self.head_dim,
#             dtype=base_attn.q_proj.weight.dtype, device=device
#         )
#         self.v_cache = torch.zeros_like(self.k_cache)

#     def forward(
#         self,
#         hidden_states: torch.Tensor,
#         attention_mask: Optional[torch.Tensor] = None,
#         position_ids: Optional[torch.LongTensor] = None,
#         past_key_value: Optional[any] = None,
#         output_attentions: bool = False,
#         use_cache: bool = False,
#         cache_position: Optional[torch.LongTensor] = None,
#         position_embeddings: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
#         **kwargs,
#     ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]: # Expected 2 return values

#         B, S, _ = hidden_states.shape
#         device = hidden_states.device

#         # 1. Resolve Position IDs and Sequence Lengths
#         if position_ids is None:
#             if cache_position is not None:
#                 position_ids = cache_position.unsqueeze(0).expand(B, -1)
#             else:
#                 position_ids = torch.arange(S, device=device).unsqueeze(0).expand(B, -1)

#         kv_lens = position_ids[:, -1] + 1

#         # 2. Linear Projections
#         q = self.base_attn.q_proj(hidden_states).view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
#         k = self.base_attn.k_proj(hidden_states).view(B, S, self.num_kv_heads, self.head_dim).transpose(1, 2)
#         v = self.base_attn.v_proj(hidden_states).view(B, S, self.num_kv_heads, self.head_dim).transpose(1, 2)

#         # 3. Apply RoPE (Crucial: do not pass position_ids if embeddings are already sliced)
#         if position_embeddings is not None:
#             cos, sin = position_embeddings
#             q, k = apply_rotary_pos_emb(q, k, cos, sin)

#         # 4. Update the Paged Cache
#         batch_indices = kwargs.get("batch_idx", torch.arange(B, device=device))
#         for b_idx in batch_indices:
#             self.paged_attn.reserve(b_idx, kv_lens[b_idx])
#         self.paged_attn.assign(batch_indices, position_ids, k, v, self.k_cache, self.v_cache)

#         # 5. Score Modification (Masking)
#         p_to_l = self.paged_attn.physical_to_logical
#         pg_size = self.paged_attn.page_size
#         # Use a safe negative value for FP16
#         NEG_INF = torch.finfo(q.dtype).min

#         def paged_score_mod(score, b, h, q_idx, kv_idx):
#             target_batch = batch_indices[b]
#             phys_block = kv_idx // pg_size
#             phys_offset = kv_idx % pg_size

#             # Map physical address back to logical position
#             logical_block = p_to_l[target_batch, phys_block]
#             logical_kv_idx = (logical_block * pg_size) + phys_offset

#             # 1. Check if the physical page belongs to this batch
#             # 2. Check if the index is within the valid sequence length
#             # 3. Apply Causal masking
#             is_valid = (logical_block >= 0) & (logical_kv_idx < kv_lens[b]) & (q_idx >= logical_kv_idx)

#             # Note: We omit indexing into attention_mask for prefill parity tests
#             # as it's typically just a causal mask already covered by is_valid.
#             return torch.where(is_valid, score, NEG_INF)

#         # 6. Execute Flex Attention
#         attn_output = flex_attention(
#             q, self.k_cache, self.v_cache,
#             score_mod=paged_score_mod,
#             scale=self.scaling,
#             enable_gqa=(self.num_heads != self.num_kv_heads)
#         )

#         # 7. Output Projection
#         attn_output = attn_output.transpose(1, 2).contiguous().view(B, S, -1)
#         output = self.base_attn.o_proj(attn_output)

#         # Return exactly 2 values: (output, attention_weights)
#         return output, None

In [5]:
import torch
from torch import nn
from torch.nn.attention.flex_attention import flex_attention
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb
from typing import Optional, Tuple

class PagedGraniteAttention(nn.Module):
    def __init__(self, base_attn, config, num_pages, page_size, max_batch_size, device='cuda'):
        super().__init__()
        self.base_attn = base_attn
        self.config = config
        self.paged_attn = PagedAttention(
            n_pages=num_pages, page_size=page_size,
            max_batch_size=max_batch_size, device=device
        )

        # Architecture Details
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.num_heads = config.num_attention_heads
        self.num_kv_heads = config.num_key_value_heads

        # 1. CRITICAL: Scale is attention_multiplier (usually 1/head_dim, e.g., 0.015625)
        # Granite uses this INSTEAD of 1/sqrt(d).
        self.scaling = getattr(config, "attention_multiplier", self.head_dim**-1)
        self.logits_soft_cap = getattr(config, "attention_logit_softcapping", None)

        # Physical Caches
        self.k_cache = torch.zeros(
            1, self.num_kv_heads, num_pages * page_size, self.head_dim,
            dtype=base_attn.q_proj.weight.dtype, device=device
        )
        self.v_cache = torch.zeros_like(self.k_cache)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[any] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
        **kwargs,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:

        B, S, _ = hidden_states.shape
        device = hidden_states.device

        # Handle position metadata
        if position_ids is None:
            position_ids = cache_position.unsqueeze(0) if cache_position is not None else torch.arange(S, device=device).unsqueeze(0).expand(B, -1)

        kv_lens = position_ids[:, -1] + 1

        # 2. Linear Projections (No manual scaling here!)
        q = self.base_attn.q_proj(hidden_states).view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.base_attn.k_proj(hidden_states).view(B, S, self.num_kv_heads, self.head_dim).transpose(1, 2)
        v = self.base_attn.v_proj(hidden_states).view(B, S, self.num_kv_heads, self.head_dim).transpose(1, 2)

        # 3. Apply RoPE (On unscaled q and k)
        if position_embeddings is not None:
            cos, sin = position_embeddings
            # apply_rotary_pos_emb handles broadcasting internally
            q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # 4. Update the Paged Cache
        batch_indices = kwargs.get("batch_idx", torch.arange(B, device=device))
        # batch_indices = kwargs.get("batch_idx", torch.arange(B, device=device))

        for i in range(B):
            self.paged_attn.reserve(int(batch_indices[i].item()), kv_lens[i])

        # for b_idx in batch_indices:
        #     self.paged_attn.reserve(b_idx, kv_lens[b_idx])
        self.paged_attn.assign(batch_indices, position_ids, k, v, self.k_cache, self.v_cache)

        # 5. Score Modification
        p_to_l = self.paged_attn.physical_to_logical
        pg_size = self.paged_attn.page_size
        NEG_INF = torch.finfo(q.dtype).min

        def paged_score_mod(score, b, h, q_idx, kv_idx):
            target_batch = batch_indices[b]
            logical_block = p_to_l[target_batch, kv_idx // pg_size]
            logical_kv_idx = (logical_block * pg_size) + (kv_idx % pg_size)

            # Mask: Unallocated page OR padding/future token OR causal boundary
            is_valid = (logical_block >= 0) & (logical_kv_idx < kv_lens[b]) & (q_idx >= logical_kv_idx)

            # Apply Softcapping if defined in config
            if self.logits_soft_cap is not None:
                score = score / self.logits_soft_cap
                score = torch.tanh(score)
                score = score * self.logits_soft_cap

            return torch.where(is_valid, score, NEG_INF)

        # 6. Execute Flex Attention with correct scaling
        # attn_output = flex_attention(
        #     q, self.k_cache, self.v_cache,
        #     score_mod=paged_score_mod,
        #     scale=self.scaling, # Use the attention_multiplier as the scale
        #     enable_gqa=(self.num_heads != self.num_kv_heads)
        # )
        attn_output = self.paged_attn.forward(
            query=q,
            k_cache=self.k_cache,
            v_cache=self.v_cache,
            kv_lens=kv_lens,
            batch_indices=batch_indices,
            score_mod=None,  # causal handled internally
            scale=self.scaling,
            enable_gqa=(self.num_heads != self.num_kv_heads),
        )


        # 7. Output Projection (Return exactly 2 values)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, S, -1)
        output = self.base_attn.o_proj(attn_output)

        return output, None

In [7]:
# def patch_granite_with_paged_attention(model, num_pages, page_size, max_batch_size, device='cuda'):
#     for layer in model.model.layers:
#           layer.self_attn = PagedGraniteAttention(
#             base_attn=layer.self_attn,
#             config=model.config,
#             num_pages=num_pages,
#             page_size=page_size,
#             max_batch_size=max_batch_size,
#             device=model.device
#         )
#     print(f"Patched all {len(model.model.layers)} attention layers.")

# # Re-run the patch
# patch_granite_with_paged_attention(model, 64, 64, 1)

In [6]:
def patch_granite_safely(model, num_pages=128, page_size=64, max_batch_size=1):
    count = 0
    for i, layer in enumerate(model.model.layers):
        # Check if it has a self_attn attribute and it's an attention class
        if hasattr(layer, "self_attn") and "Attention" in type(layer.self_attn).__name__:
            layer.self_attn = PagedGraniteAttention(
                base_attn=layer.self_attn,
                config=model.config,
                num_pages=num_pages,
                page_size=page_size,
                max_batch_size=max_batch_size,
                device="cuda"
            )
            count += 1
    print(f"Successfully patched {count} attention layers. Skipped Mamba layers.")

# Usage:
patch_granite_safely(model)

Successfully patched 40 attention layers. Skipped Mamba layers.


In [7]:
torch.compile(model)

OptimizedModule(
  (_orig_mod): GraniteMoeHybridForCausalLM(
    (model): GraniteMoeHybridModel(
      (embed_tokens): Embedding(100352, 2560, padding_idx=100256)
      (layers): ModuleList(
        (0-39): 40 x GraniteMoeHybridDecoderLayer(
          (input_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
          (post_attention_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
          (shared_mlp): GraniteMoeHybridMLP(
            (activation): SiLUActivation()
            (input_linear): Linear(in_features=2560, out_features=16384, bias=False)
            (output_linear): Linear(in_features=8192, out_features=2560, bias=False)
          )
          (self_attn): PagedGraniteAttention(
            (base_attn): GraniteMoeHybridAttention(
              (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
              (k_proj): Linear(in_features=2560, out_features=512, bias=False)
              (v_proj): Linear(in_features=2560, out_features=512, bias=Fals

In [8]:
import torch
from transformers import AutoTokenizer
def test_parity(model_original, model_patched, text="Hello, my name is IBM Granite."):
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-4.0-micro")
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Run original model
    with torch.no_grad():
        out_orig = model_original(**inputs).logits

    # Run patched model
    # (Note: You might need to manually pass batch_idx if your forward expects it)
    with torch.no_grad():
        # Clear/Reset your paged cache before this call if necessary
        out_patched = model_patched(**inputs).logits

    # Calculate difference
    diff = torch.abs(out_orig - out_patched).max().item()
    print(f"Max Absolute Difference: {diff}")

    if diff < 1e-3:
        print("✅ SUCCESS: The outputs match!")
    else:
        print("❌ FAILURE: The outputs differ significantly. Check RoPE or Masking.")

# Usage:
model_orig = AutoModelForCausalLM.from_pretrained(
    "ibm-granite/granite-4.0-micro",
    torch_dtype=torch.float16,
    device_map="cuda"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
import torch
from transformers import AutoTokenizer
import torch.nn.functional as F

def detailed_parity_test(model_original, model_patched, text="Hello, my name is IBM Granite."):
    tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-4.0-micro")
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    print("=" * 80)
    print("DIAGNOSTIC TEST START")
    print("=" * 80)

    # ------------------------------------------------------------------
    # TEST 0: Paged vs normal KV cache check  ✅ NEW
    # ------------------------------------------------------------------
    for i, layer in enumerate(model_patched.model.layers):
        attn = layer.self_attn

        # 1️⃣ Ensure normal HF KV cache is not present
        assert not hasattr(attn, "past_key_value"), f"❌ Layer {i}: normal past_key_value exists"

        # 2️⃣ Ensure paged attention cache exists
        assert hasattr(attn, "paged_attn"), f"❌ Layer {i}: paged_attn missing"
        assert hasattr(attn, "k_cache") and hasattr(attn, "v_cache"), f"❌ Layer {i}: paged K/V cache missing"

    print("✅ Paged attention cache confirmed. Normal KV cache disabled.")

    # ------------------------------------------------------------------
    # TEST 1: Model modes
    # ------------------------------------------------------------------
    print("\nTEST 1: Model Modes")
    print(f"Original training mode: {model_original.training}")
    print(f"Patched training mode: {model_patched.training}")

    # ------------------------------------------------------------------
    # TEST 2: Embedding parity
    # ------------------------------------------------------------------
    with torch.no_grad():
        hidden_orig = model_original.model.embed_tokens(inputs["input_ids"])
        hidden_patched = model_patched.model.embed_tokens(inputs["input_ids"])

    print("\nTEST 2: Embedding Parity")
    print(f"Max embedding diff: {(hidden_orig - hidden_patched).abs().max().item():.6e}")

    # ------------------------------------------------------------------
    # TEST 3: QKV weight parity
    # ------------------------------------------------------------------
    layer_idx = 0
    orig_attn = model_original.model.layers[layer_idx].self_attn
    patched_attn = model_patched.model.layers[layer_idx].self_attn

    print("\nTEST 3: Projection Weight Parity")
    print(f"Q diff: {(orig_attn.q_proj.weight - patched_attn.base_attn.q_proj.weight).abs().max().item():.6e}")
    print(f"K diff: {(orig_attn.k_proj.weight - patched_attn.base_attn.k_proj.weight).abs().max().item():.6e}")
    print(f"V diff: {(orig_attn.v_proj.weight - patched_attn.base_attn.v_proj.weight).abs().max().item():.6e}")
    print(f"O diff: {(orig_attn.o_proj.weight - patched_attn.base_attn.o_proj.weight).abs().max().item():.6e}")

    # ------------------------------------------------------------------
    # TEST 4: QKV activation parity
    # ------------------------------------------------------------------
    with torch.no_grad():
        q_o = orig_attn.q_proj(hidden_orig)
        k_o = orig_attn.k_proj(hidden_orig)
        v_o = orig_attn.v_proj(hidden_orig)

        q_p = patched_attn.base_attn.q_proj(hidden_patched)
        k_p = patched_attn.base_attn.k_proj(hidden_patched)
        v_p = patched_attn.base_attn.v_proj(hidden_patched)

    print("\nTEST 4: QKV Activation Parity")
    print(f"Q diff: {(q_o - q_p).abs().max().item():.6e}")
    print(f"K diff: {(k_o - k_p).abs().max().item():.6e}")
    print(f"V diff: {(v_o - v_p).abs().max().item():.6e}")

    # ------------------------------------------------------------------
    # RESET CACHE
    # ------------------------------------------------------------------
    for layer in model_patched.model.layers:
        attn = layer.self_attn
        attn.k_cache.zero_()
        attn.v_cache.zero_()
        attn.paged_attn.page_table.fill_(-1)
        attn.paged_attn.capacity.zero_()
        attn.paged_attn.physical_to_logical.fill_(-1)
        attn.paged_attn.empty_pages = list(range(attn.paged_attn.n_pages - 1, -1, -1))

    # ------------------------------------------------------------------
    # TEST 5: Full forward pass
    # ------------------------------------------------------------------
    print("\nTEST 5: Full Forward Pass")

    with torch.no_grad():
        out_orig = model_original(**inputs).logits
        out_patched = model_patched(**inputs).logits

    # ------------------------------------------------------------------
    # TEST 6: POST-FORWARD cache validation
    # ------------------------------------------------------------------
    print("\nTEST 6: Post-Forward Cache State")

    attn0 = model_patched.model.layers[0].self_attn
    print(f"K cache nonzero: {(attn0.k_cache != 0).sum().item()}")
    print(f"V cache nonzero: {(attn0.v_cache != 0).sum().item()}")
    print(f"Capacity: {attn0.paged_attn.capacity.item()}")
    print(f"Pages allocated: {(attn0.paged_attn.page_table >= 0).sum().item()}")

    # ------------------------------------------------------------------
    # TEST 7: Output difference
    # ------------------------------------------------------------------
    diff = (out_orig - out_patched).abs()

    print("\nTEST 7: Output Difference")
    print(f"Max diff: {diff.max().item():.6f}")
    print(f"Mean diff: {diff.mean().item():.6f}")

    # ------------------------------------------------------------------
    # VERDICT
    # ------------------------------------------------------------------
    print("\nDIAGNOSIS")

    max_diff = diff.max().item()

    if max_diff < 0.2:
        print("✅ PASS: fp16-consistent output (expected drift)")
    else:
        print("❌ FAIL: Unexpected numerical divergence")

    return max_diff


In [10]:
def reset_model_cache(model):
    for layer in model.model.layers:
        if hasattr(layer.self_attn, "paged_attn"):
            pa = layer.self_attn.paged_attn
            pa.empty_pages = list(range(pa.n_pages - 1, -1, -1))
            pa.capacity.zero_()
            pa.page_table.fill_(-1)
            pa.physical_to_logical.fill_(-1)
            layer.self_attn.k_cache.zero_()
            layer.self_attn.v_cache.zero_()

# Usage:
reset_model_cache(model)
detailed_parity_test(model_orig, model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

DIAGNOSTIC TEST START
✅ Paged attention cache confirmed. Normal KV cache disabled.

TEST 1: Model Modes
Original training mode: False
Patched training mode: False

TEST 2: Embedding Parity
Max embedding diff: 0.000000e+00

TEST 3: Projection Weight Parity
Q diff: 0.000000e+00
K diff: 0.000000e+00
V diff: 0.000000e+00
O diff: 0.000000e+00


GraniteMoeHybrid requires an initialized `HybridMambaAttentionDynamicCache` to return a cache. Because one was not provided, no cache will be returned.



TEST 4: QKV Activation Parity
Q diff: 0.000000e+00
K diff: 0.000000e+00
V diff: 0.000000e+00

TEST 5: Full Forward Pass

TEST 6: Post-Forward Cache State
K cache nonzero: 4096
V cache nonzero: 4096
Capacity: 64
Pages allocated: 1

TEST 7: Output Difference
Max diff: 0.156250
Mean diff: 0.009895

DIAGNOSIS
✅ PASS: fp16-consistent output (expected drift)


0.15625

In [11]:
def test_generation(model, tokenizer, prompt="The Future of Ai is"):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # We use a simple loop because model.generate often expects
    # specific KV cache objects that we've replaced
    generated = inputs["input_ids"]

    for i in range(20):
        with torch.no_grad():
            # Ensure position_ids are correct for the current length
            outputs = model(input_ids=generated, position_ids=torch.arange(generated.shape[1]).unsqueeze(0).cuda())
            next_token_logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            generated = torch.cat([generated, next_token], dim=-1)

            print(tokenizer.decode(next_token[0]), end="", flush=True)
    print("\n")

# If it outputs " Paris. It is a beautiful city..." -> SUCCESS
# If it outputs " !!!77777$$$ " -> FAILURE (usually RoPE is wrong)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-4.0-micro")
test_generation(model,tokenizer)

 Here

The future of AI is here, and it's already transforming the way we live, work



## Benchmark Evaluation


In [15]:
DEVICE = "cuda"
MODEL_NAME = "ibm-granite/granite-4.0-micro"
MAX_NEW_TOKENS = 64
PAGE_SIZE = 64
TOTAL_PAGES = 1024

In [16]:
from datasets import load_dataset

print("Loading datasets...")
mmlu = load_dataset("cais/mmlu", "abstract_algebra", split="validation", streaming=True).take(5)
truthfulqa = load_dataset("domenicrosati/TruthfulQA", split="train", streaming=True).take(5)


Loading datasets...


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [17]:
def run_benchmark(model, prompts, label):
    results = []

    for i, text in enumerate(prompts):
        inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
        input_len = inputs.input_ids.shape[1]


        # Warmup (1 run)
        with torch.no_grad():
            _ = model.generate(**inputs, max_new_tokens=5, do_sample=False)

        # Measure
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)

        start_event.record()
        with torch.no_grad():
            # Use generate to simulate real inference
            output = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                use_cache=True # Original uses standard cache, Paged uses its internal
            )
        end_event.record()
        torch.cuda.synchronize()

        latency = start_event.elapsed_time(end_event) / 1000 # Seconds
        throughput = MAX_NEW_TOKENS / latency
        memory = torch.cuda.max_memory_allocated() / 1024**2 # MB

        results.append({
            "Label": label,
            "Sample": i,
            "Input_Len": input_len,
            "Latency": latency,
            "Throughput": throughput,
            "Memory": memory
        })
        print(f"[{label}] Sample {i} | Memory: {memory:.1f}MB | Throughput: {throughput:.2f} t/s")

    return results

In [19]:
all_prompts = [x['question'] for x in mmlu] + [x['Question'] for x in truthfulqa]

print("\nStarting Baseline Benchmark...")
orig_results = run_benchmark(model_orig, all_prompts, "Original (SDPA)")

print("\nStarting Paged Attention Benchmark...")
paged_results = run_benchmark(model, all_prompts, "Paged (Flex)")

KeyError: 'Question'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.DataFrame(orig_results + paged_results)

# Create Professional Comparison Plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Memory Usage
for label in df['Label'].unique():
    subset = df[df['Label'] == label]
    ax1.bar(subset['Sample'] + (0.2 if "Paged" in label else -0.2),
            subset['Memory'], width=0.4, label=label)
ax1.set_title("Peak GPU Memory Comparison")
ax1.set_ylabel("Memory (MB)")
ax1.set_xlabel("Sample Index")
ax1.legend()

# Plot 2: Throughput
for label in df['Label'].unique():
    subset = df[df['Label'] == label]
    ax2.plot(subset['Sample'], subset['Throughput'], marker='o', label=label)
ax2.set_title("Decoding Throughput (higher is better)")
ax2.set_ylabel("Tokens / Second")
ax2.set_xlabel("Sample Index")
ax2.legend()

plt.tight_layout()
plt.savefig("paged_attention_benchmark.png")
plt.show()

print("\nBenchmark Complete. Results saved to 'paged_attention_benchmark.png'")

## Evaluation

In [ ]:
"""
Granite 4 + DeepEval MMLU + TruthfulQA

Metrics:
  accuracy, latency (ms/query), throughput, GPU memory,
  batching efficiency, context length scalability
"""

import os
import re
import time
import math
import random
import statistics
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Type

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset, DownloadConfig
from deepeval.models import DeepEvalBaseLLM
from deepeval.benchmarks import MMLU, TruthfulQA
from deepeval.benchmarks.mmlu.task import MMLUTask
from deepeval.benchmarks.tasks import TruthfulQATask
from deepeval.benchmarks.modes import TruthfulQAMode


# Hugging Face home Directory
#___________________
HF_CACHE_DIR = os.environ.get("HF_HOME", os.path.expanduser("~/.cache/huggingface"))
os.environ.setdefault("HF_HOME", HF_CACHE_DIR)
os.environ.setdefault("HF_DATASETS_CACHE", os.path.join(HF_CACHE_DIR, "datasets"))

def load_dataset_with_retry(
    path: str,
    name: Optional[str] = None,
    split: Optional[str] = None,
    retries: int = 6,
    base_delay: float = 2.0,
    **kwargs,
):
    """
    Load hugging face dataset. Added try catch to prevent download/access errors
    """
    download_config = kwargs.pop("download_config", None) or DownloadConfig(
        resume_download=True,
        max_retries=1,
    )

    last_err = None
    for attempt in range(retries):
        try:
            return load_dataset(
                path,
                name=name,
                split=split,
                download_config=download_config,
                **kwargs,
            )
        except Exception as e:
            last_err = e
            msg = str(e)
            transient = any(s in msg for s in [
                " 502 ", " 503 ", " 504 ",
                "Bad Gateway", "Service Unavailable", "Gateway",
                "Read timed out", "Connection reset", "Temporary failure",
                "MaxRetryError", "HTTPSConnectionPool",
            ])
            if not transient and attempt >= 1:
                raise
            sleep_s = base_delay * (2 ** attempt) + random.random()
            print(f"[dataset retry] attempt {attempt+1}/{retries} failed: {e}\n -> sleeping {sleep_s:.1f}s")
            time.sleep(sleep_s)
    raise last_err


#helper functions
#_____________________
def _cuda_sync_if_needed(device: str):
    """
    use syncronize to ensure accuracy
    """
    if device.startswith("cuda") and torch.cuda.is_available():
        torch.cuda.synchronize()

def _percentile(xs: List[float], p: float) -> float:
    """
    calculate the percentile with linear interpolation
    """
    if not xs:
        return float("nan")
    xs_sorted = sorted(xs)
    k = (len(xs_sorted) - 1) * p
    f = math.floor(k)
    c = math.ceil(k)
    if f == c:
        return xs_sorted[int(k)]
    return xs_sorted[f] * (c - k) + xs_sorted[c] * (k - f)

def _format_bytes(n: Optional[float]) -> str:
    """
    format bytes to to string representation
    """
    if n is None or (isinstance(n, float) and math.isnan(n)):
        return "n/a"
    units = ["B", "KB", "MB", "GB", "TB"]
    u = 0
    n = float(n)
    while n >= 1024 and u < len(units) - 1:
        n /= 1024
        u += 1
    return f"{n:.2f} {units[u]}"


#metrics
#______________
@dataclass
class CallMetrics:
    latency_ms: float
    input_tokens: int
    output_tokens: int
    peak_alloc_bytes: Optional[int] = None
    peak_reserved_bytes: Optional[int] = None

@dataclass
class RunSummary:
    n_calls: int
    avg_latency_ms: float
    p50_latency_ms: float
    p95_latency_ms: float
    qps: float
    tok_per_s_in: float
    tok_per_s_out: float
    peak_alloc_bytes_max: Optional[int]
    peak_reserved_bytes_max: Optional[int]

def summarize_calls(calls: List[CallMetrics]) -> RunSummary:
    n = len(calls)
    lats = [c.latency_ms for c in calls]
    in_toks = [c.input_tokens for c in calls]
    out_toks = [c.output_tokens for c in calls]
    total_s = sum(lats) / 1000.0 if lats else 0.0

    peak_allocs = [c.peak_alloc_bytes for c in calls if c.peak_alloc_bytes is not None]
    peak_resvs = [c.peak_reserved_bytes for c in calls if c.peak_reserved_bytes is not None]

    return RunSummary(
        n_calls=n,
        avg_latency_ms=(statistics.mean(lats) if lats else float("nan")),
        p50_latency_ms=_percentile(lats, 0.50),
        p95_latency_ms=_percentile(lats, 0.95),
        qps=(n / total_s if total_s > 0 else float("nan")),
        tok_per_s_in=(sum(in_toks) / total_s if total_s > 0 else float("nan")),
        tok_per_s_out=(sum(out_toks) / total_s if total_s > 0 else float("nan")),
        peak_alloc_bytes_max=(max(peak_allocs) if peak_allocs else None),
        peak_reserved_bytes_max=(max(peak_resvs) if peak_resvs else None),
    )


# =========================
# DeepEval "schema" return shim
# =========================
class AnswerObj:
    """Minimal object compatible with DeepEval expecting `.answer`."""
    def __init__(self, answer: str):
        self.answer = answer


# =========================
# Granite wrapper (structured schema + single-device model)
# =========================
class Granite4HF(DeepEvalBaseLLM):
    def __init__(
        self,
        model_path: str = "ibm-granite/granite-4.0-micro",
        device: str = "cuda",
        max_new_tokens: int = 16,
        dtype: Optional[torch.dtype] = None,
    ):
        self.model_path = model_path
        self.device = device
        self.max_new_tokens = max_new_tokens

        self._is_cuda = device.startswith("cuda") and torch.cuda.is_available()

        if dtype is None:
            dtype = torch.float16 if self._is_cuda else torch.float32
        self.dtype = dtype

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Prefer dtype=... (newer); fall back to torch_dtype=... (older)
        try:
            self.model = AutoModelForCausalLM.from_pretrained(model_path, dtype=self.dtype)
        except TypeError:
            self.model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=self.dtype)

        self.model.to(device)
        self.model.eval()

        self.calls: List[CallMetrics] = []
        if self._is_cuda:
            torch.cuda.reset_peak_memory_stats()

    def reset_metrics(self):
        self.calls.clear()
        if self._is_cuda:
            torch.cuda.reset_peak_memory_stats()

    def get_model_name(self) -> str:
        return self.model_path

    def load_model(self):
        return self.model

    def _extract_choice_token(self, text: str) -> str:
        """
        For MC tasks, DeepEval wants a compact option token.
        We try:
          - A-E
          - A-D (common)
          - otherwise return stripped text
        """
        t = text.strip()
        m = re.search(r"\b([A-E])\b", t)
        return m.group(1) if m else t

    @torch.inference_mode()
    def generate(self, prompt: Optional[str] = None, **kwargs):
        """
        DeepEval may call:
          generate(prompt=..., schema=MultipleChoiceSchema)
        If `schema` is provided, return an object with `.answer`.
        Otherwise return a plain string.
        """
        if prompt is None:
            prompt = kwargs.get("prompt", "")

        schema: Optional[Type] = kwargs.get("schema", None)

        if self._is_cuda:
            torch.cuda.reset_peak_memory_stats()

        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_len = int(inputs["input_ids"].shape[-1])
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        _cuda_sync_if_needed(self.device)
        t0 = time.perf_counter()

        out = self.model.generate(
            **inputs,
            max_new_tokens=self.max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            use_cache=True,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
        )

        _cuda_sync_if_needed(self.device)
        t1 = time.perf_counter()

        gen_text = self.tokenizer.decode(out[0], skip_special_tokens=True)
        output_len = int(out.shape[-1])
        new_tokens = max(0, output_len - input_len)

        completion = gen_text[len(prompt):] if len(gen_text) >= len(prompt) else gen_text
        answer = self._extract_choice_token(completion)

        latency_ms = (t1 - t0) * 1000.0
        peak_alloc = int(torch.cuda.max_memory_allocated()) if self._is_cuda else None
        peak_resv = int(torch.cuda.max_memory_reserved()) if self._is_cuda else None

        self.calls.append(
            CallMetrics(
                latency_ms=latency_ms,
                input_tokens=input_len,
                output_tokens=new_tokens,
                peak_alloc_bytes=peak_alloc,
                peak_reserved_bytes=peak_resv,
            )
        )

        # If DeepEval requested a schema, return an object with `.answer`
        if schema is not None:
            # Some DeepEval paths might accept a schema class instance; we keep it minimal.
            return AnswerObj(answer)

        return answer

    async def a_generate(self, prompt: str, **kwargs):
        return self.generate(prompt=prompt, **kwargs)


#MMLU
#__________________
class MMLU_Retry(MMLU):
    def load_benchmark_dataset(self, task: MMLUTask):
        self.dataset = load_dataset_with_retry("cais/mmlu", name=task.value)
        return super().load_benchmark_dataset(task)


@torch.inference_mode()
def bench_batching_efficiency(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    device: str,
    base_prompt: str,
    batch_sizes: List[int],
    max_new_tokens: int = 16,
    warmup: int = 1,
    iters: int = 5,
) -> List[Dict[str, Any]]:
    is_cuda = device.startswith("cuda") and torch.cuda.is_available()
    results = []

    for bs in batch_sizes:
        prompts = [base_prompt] * bs
        enc = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
        enc = {k: v.to(device) for k, v in enc.items()}

        for _ in range(warmup):
            _cuda_sync_if_needed(device)
            _ = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            _cuda_sync_if_needed(device)

        lat_ms_runs = []
        out_tokens_runs = []
        in_tokens_total = int(enc["input_ids"].numel())

        for _ in range(iters):
            if is_cuda:
                torch.cuda.reset_peak_memory_stats()

            _cuda_sync_if_needed(device)
            t0 = time.perf_counter()

            out = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

            _cuda_sync_if_needed(device)
            t1 = time.perf_counter()

            lat_ms = (t1 - t0) * 1000.0
            lat_ms_runs.append(lat_ms)

            out_tokens_total = int(out.numel()) - int(enc["input_ids"].numel())
            out_tokens_runs.append(max(0, out_tokens_total))

        avg_lat_ms = statistics.mean(lat_ms_runs)
        total_s = avg_lat_ms / 1000.0

        results.append(
            {
                "batch_size": bs,
                "avg_batch_latency_ms": avg_lat_ms,
                "latency_ms_per_query": avg_lat_ms / bs,
                "qps": (bs / total_s) if total_s > 0 else float("nan"),
                "in_tokens_per_s": (in_tokens_total / total_s) if total_s > 0 else float("nan"),
                "out_tokens_per_s": (statistics.mean(out_tokens_runs) / total_s) if total_s > 0 else float("nan"),
                "peak_alloc": int(torch.cuda.max_memory_allocated()) if is_cuda else None,
                "peak_reserved": int(torch.cuda.max_memory_reserved()) if is_cuda else None,
            }
        )

    return results


@torch.inference_mode()
def bench_context_length_scalability(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    device: str,
    token_lengths: List[int],
    max_new_tokens: int = 8,
    warmup: int = 1,
    iters: int = 5,
) -> List[Dict[str, Any]]:
    is_cuda = device.startswith("cuda") and torch.cuda.is_available()
    results = []
    chunk = " The quick brown fox jumps over the lazy dog."

    for target_len in token_lengths:
        prompt = ""
        while len(tokenizer(prompt).input_ids) < target_len:
            prompt += chunk

        enc = tokenizer(prompt, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}
        in_len = int(enc["input_ids"].shape[-1])

        for _ in range(warmup):
            _cuda_sync_if_needed(device)
            _ = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            _cuda_sync_if_needed(device)

        lats = []
        for _ in range(iters):
            if is_cuda:
                torch.cuda.reset_peak_memory_stats()

            _cuda_sync_if_needed(device)
            t0 = time.perf_counter()

            _ = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

            _cuda_sync_if_needed(device)
            t1 = time.perf_counter()
            lats.append((t1 - t0) * 1000.0)

        results.append(
            {
                "input_tokens": in_len,
                "avg_latency_ms": statistics.mean(lats),
                "p95_latency_ms": _percentile(lats, 0.95),
                "ms_per_input_token": statistics.mean(lats) / max(1, in_len),
                "peak_alloc": int(torch.cuda.max_memory_allocated()) if is_cuda else None,
                "peak_reserved": int(torch.cuda.max_memory_reserved()) if is_cuda else None,
            }
        )

    return results


def print_run_report(title: str, overall_score: float, calls: List[CallMetrics]):
    s = summarize_calls(calls)
    print(f"\n=== {title} ===")
    print(f"Accuracy (overall_score): {overall_score:.4f}")
    print(f"Queries: {s.n_calls}")
    print(f"Latency (ms/query): avg={s.avg_latency_ms:.2f}, p50={s.p50_latency_ms:.2f}, p95={s.p95_latency_ms:.2f}")
    print(f"Throughput: {s.qps:.2f} queries/s")
    print(f"Token throughput: in={s.tok_per_s_in:.2f} tok/s, out={s.tok_per_s_out:.2f} tok/s")
    print(f"Peak GPU memory (allocated): {_format_bytes(s.peak_alloc_bytes_max)}")
    print(f"Peak GPU memory (reserved):  {_format_bytes(s.peak_reserved_bytes_max)}")

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "ibm-granite/granite-4.0-h-1b"

granite = Granite4HF(model_path=model_path, device=device, max_new_tokens=16)

# -------- MMLU --------
granite.reset_metrics()
mmlu = MMLU_Retry(
    tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY],
    n_shots=3,
)
mmlu.evaluate(model=granite)
print_run_report("MMLU Results + Metrics", mmlu.overall_score, granite.calls)

# ----- TruthfulQA -----
granite.reset_metrics()
truthfulqa = TruthfulQA(
    tasks=[TruthfulQATask.SCIENCE, TruthfulQATask.FINANCE],
    mode=TruthfulQAMode.MC1,
)
truthfulqa.evaluate(model=granite)
print_run_report("TruthfulQA Results + Metrics", truthfulqa.overall_score, granite.calls)

# -------- Optional microbenchmarks --------
hf_model = granite.model
hf_tok = granite.tokenizer

print("\n=== Batching Efficiency Microbenchmark ===")
base_prompt = (
    "Answer with ONLY one letter: A, B, C, or D.\n"
    "Question: Which planet is known as the Red Planet?\n"
    "A) Venus\nB) Mars\nC) Jupiter\nD) Mercury\n"
    "Answer:"
)
batch_results = bench_batching_efficiency(
    model=hf_model,
    tokenizer=hf_tok,
    device=device,
    base_prompt=base_prompt,
    batch_sizes=[1, 2, 4, 8, 16],
    max_new_tokens=8,
    warmup=1,
    iters=5,
)
for r in batch_results:
    print(
        f"BS={r['batch_size']:>2} | ms/query={r['latency_ms_per_query']:.2f} | "
        f"qps={r['qps']:.2f} | in_tok/s={r['in_tokens_per_s']:.0f} | out_tok/s={r['out_tokens_per_s']:.0f} | "
        f"peak_alloc={_format_bytes(r['peak_alloc'])}"
    )

print("\n=== Context Length Scalability Microbenchmark ===")
max_ctx = getattr(getattr(hf_model, "config", None), "max_position_embeddings", None) or 4096
token_lengths = [t for t in [128, 256, 512, 1024, 2048, 4096] if t <= max_ctx]

ctx_results = bench_context_length_scalability(
    model=hf_model,
    tokenizer=hf_tok,
    device=device,
    token_lengths=token_lengths,
    max_new_tokens=8,
    warmup=1,
    iters=5,
)
print(f"Using max_position_embeddings={max_ctx}")
for r in ctx_results:
    print(
        f"in_tokens={r['input_tokens']:>4} | avg_ms={r['avg_latency_ms']:.2f} | p95_ms={r['p95_latency_ms']:.2f} | "
        f"ms/token={r['ms_per_input_token']:.4f} | peak_alloc={_format_bytes(r['peak_alloc'])}"
    )

## Scripts

In [ ]:
! python tests.py